In [3]:
import pandas as pd
import numpy as np
import os
import altair as alt
import eco_style
import requests
import json
alt.themes.enable("light")

ThemeRegistry.enable('light')

In [4]:
image = alt.Chart(pd.DataFrame([{'url': 'https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/refs/heads/main/guidelines/logos/eco-logo-light.png'}])).mark_image(
    height=30,
    width=90,
    opacity=0.5,
    # align='right',
    baseline='bottom'
).encode(
    url='url',
    x=alt.value(400),
    y=alt.value(0)
)

# IIP

In [3]:
df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")

df = df.query("INDICATOR == 'International investment position, Net (assets less liabilities), Adjusted using IMF accounting records'")
df = df.query("TIME_PERIOD == 2023")
targets = ["United States", "United Kingdom"]
df = df.query("COUNTRY in @targets")

gdp_df = pd.read_csv("API_NY/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_26433.csv", skiprows=4)
gdp_df = gdp_df.query("`Country Name` in @targets")
gdp_df = gdp_df[["Country Name", "Country Code", "2023"]]
gdp_df.columns = ["COUNTRY", "COUNTRY_CODE", "GDP"]

df = df.merge(gdp_df, on="COUNTRY", how="left")
df['value_pct'] = df['OBS_VALUE'] / df['GDP']
df['value_nominal'] = df['OBS_VALUE'] / 1e9

# Nominal

base = alt.Chart(df).mark_bar(
    color="#36b7b4"
).encode(
    x=alt.X("value_nominal:Q", title="", axis=alt.Axis(
        format="$,.0f", labelExpr="datum.label + ' bn'")),
    y=alt.Y("COUNTRY:N", 
            axis=alt.Axis(
                domain=False,
                orient="right",),
            title=""),
)


chart = (base+image.encode(x=alt.value(50), y=alt.value(35))
         
         ).properties(
    width=500,
    height=250,
)

chart.save("charts/nominal_iip.png", scale_factor=3)
chart.save("charts/nominal_iip.json")

chart.display()

# Percentage of GDP

base = alt.Chart(df).mark_bar(
    color="#36b7b4"
).encode(
    x=alt.X("value_pct:Q", title="", axis=alt.Axis(
        
        labelExpr="datum.label + (datum.value == 0 ? ' of GDP ' : '')",
        labelAlign=alt.expr("datum.value == 0 ? 'left' : 'center'"),
        format=",.0%",)),
    y=alt.Y("COUNTRY:N", 
            axis=alt.Axis(
                domain=False,
                orient="right",),
            title=""),
)

chart = (base+image.encode(x=alt.value(50), y=alt.value(35))).properties(
    width=500,
    height=200,
)

chart.save("charts/percentage_iip.png", scale_factor=3)
chart.save("charts/percentage_iip.json")
chart.display()




/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_95849/2192272792.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")


alt.LayerChart(...)

alt.LayerChart(...)

In [4]:
df

,COUNTRY,INDICATOR,TYPE_OF_TRANSFORMATION,FREQUENCY,TIME_PERIOD,OBS_VALUE,SCALE,PRECISION,DECIMALS_DISPLAYED,INSTR_ASSET,...,SHORT_SOURCE_CITATION,FULL_SOURCE_CITATION,LICENSE,SUGGESTED_CITATION,KEY_INDICATOR,SERIES_NAME,COUNTRY_CODE,GDP,value_pct,value_nominal
0,United States,"International investment position, Net (assets...",US dollar,Annual,2023,-1.985315e+13,Millions,Millions,NaN,NaN,...,Country authorities; National Summary Data Pag...,NaN,© International Monetary Fund Copyright. All R...,International Monetary Fund. Balance of Paymen...,NaN,NaN,USA,2.772071e+13,-0.716185,-19853.153375
1,United Kingdom,"International investment position, Net (assets...",US dollar,Annual,2023,-8.381277e+11,Millions,Millions,NaN,NaN,...,Country authorities; National Summary Data Pag...,NaN,© International Monetary Fund Copyright. All R...,International Monetary Fund. Balance of Paymen...,NaN,NaN,GBR,3.380855e+12,-0.247904,-838.127657


In [5]:
gdp_df = pd.read_csv("API_NY/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_26433.csv", skiprows=4)
gdp_df=gdp_df[["Country Name", "Country Code", "2022"]]
# What's lebanon's GDP? 2.099242e+10
# Who has the closest gdp to lebanon?
gdp_df = gdp_df.dropna()
gdp_df['diff'] = np.abs(gdp_df['2022'] - 2.099242e+10)
gdp_df['pct_diff'] = gdp_df['diff'] / gdp_df['2022']
gdp_df = gdp_df.sort_values('diff', ascending=True)
gdp_df.head(20)

,Country Name,Country Code,2022,diff,pct_diff
130,Lebanon,LBN,2.099242e+10,1.948808e+03,9.283389e-08
80,Gabon,GAB,2.044066e+10,5.517643e+08,2.699347e-02
33,Botswana,BWA,2.032196e+10,6.704611e+08,3.299195e-02
100,Haiti,HTI,2.025355e+10,7.388681e+08,3.648091e-02
85,Guinea,GIN,1.991045e+10,1.081967e+09,5.434168e-02
10,Armenia,ARM,1.951351e+10,1.478913e+09,7.578922e-02
159,Malta,MLT,1.920998e+10,1.782440e+09,9.278719e-02
196,West Bank and Gaza,PSE,1.916550e+10,1.826920e+09,9.532337e-02
5,Albania,ALB,1.901724e+10,1.975177e+09,1.038625e-01
165,Mozambique,MOZ,1.888353e+10,2.108891e+09,1.116789e-01


# Charts: Lebanon's Current Account

In [6]:
gdp_df

,Country Name,Country Code,2022,diff,pct_diff
130,Lebanon,LBN,2.099242e+10,1.948808e+03,9.283389e-08
80,Gabon,GAB,2.044066e+10,5.517643e+08,2.699347e-02
33,Botswana,BWA,2.032196e+10,6.704611e+08,3.299195e-02
100,Haiti,HTI,2.025355e+10,7.388681e+08,3.648091e-02
85,Guinea,GIN,1.991045e+10,1.081967e+09,5.434168e-02
...,...,...,...,...,...
103,IDA & IBRD total,IBT,4.059681e+13,4.057582e+13,9.994829e-01
198,Post-demographic dividend,PST,5.706675e+13,5.704576e+13,9.996321e-01
181,OECD members,OED,6.069133e+13,6.067034e+13,9.996541e-01
95,High income,HIC,6.496126e+13,6.494027e+13,9.996768e-01


In [7]:
import pandas as pd
import altair as alt

# Load dataset
df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")

# Focus on 2023 and your target countries
df = df.query("TIME_PERIOD == 2023 and `TYPE_OF_TRANSFORMATION` == 'US dollar'")
targets = ["Botswana", "Armenia, Republic of", "Lebanon"]
df = df[df["COUNTRY"].isin(targets)]

# Indicators for current account components
components = {
    "Goods": "Goods balance (credit less debit), Net (credits less debits)",
    "Services": "Services balance (credit less debit), Net (credits less debits)",
    "Primary income": "Primary income balance (credit less debit), Net (credits less debits)",
    "Secondary income": "Secondary income balance (credit less debit), Net (credits less debits)",
    "Current account balance": "Current account balance (credit less debit), Net (credits less debits)"
}

df = df[df["INDICATOR"].isin(components.values())]
df["Component"] = df["INDICATOR"].map({v: k for k, v in components.items()})

# Convert to USD billions
df["Value (USD bn)"] = df["OBS_VALUE"] / 1e9

# Chart
bars = alt.Chart(df.query("Component != 'Current account balance'")).mark_bar().encode(
                     x=alt.X("Value (USD bn):Q", title="", axis=alt.Axis(format="$,.0f", labelExpr="datum.label + ' bn'")),
    y=alt.Y("COUNTRY:N", title="", axis=alt.Axis(orient="left", domain=False)),
    color=alt.Color("Component:N",
        legend=alt.Legend(orient="top"),
        scale=alt.Scale(
            range=["#00A767","#36B7B4","#179FDB","#0063AF","#243B5A"],
            domain=list(components.keys())),
         title=""),
    tooltip=["COUNTRY", "Component", "Value (USD bn)"]
).properties(
    width=500,
    height=250,
    title=""
)

# balance points
balance = alt.Chart(df.query("Component == 'Current account balance'")).mark_point(
    color="black",
    size=100,
).encode(
    x=alt.X("Value (USD bn):Q", title="", axis=alt.Axis(format="$,.0f")),
    y=alt.Y("COUNTRY:N", title="", axis=alt.Axis(orient="left")),
)

chart = bars + balance + image.encode(x=alt.value(450), y=alt.value(-50))

chart.save("charts/leb_current_account_components.png", scale_factor=3)
chart.save("charts/leb_current_account_components.json")

/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_95849/3965007630.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")


# Remittances: Lebanon

In [8]:
df = pd.read_csv("API_BX/API_BX.TRF.PWKR.DT.GD.ZS_DS2_en_csv_v2_14332.csv", skiprows=4)
targets = ["Lebanon"] # "Jordan", "Tunisia", "Georgia"]
df = df.query("`Country Name` in @targets")
df = df.drop(columns=["Country Code", "Indicator Name", "Indicator Code"])
df = df.melt(id_vars=["Country Name"], var_name="Year", value_name="Value")
df = df.dropna()
df['Value'] = df['Value'].astype(float)/100

df['end_label'] = ''
df.iloc[-1, df.columns.get_loc('end_label')] = "Lebanese remittances\n(% of GDP)"
df['end_label'] = df['end_label'].str.split("\n")

chart = alt.Chart(df).mark_line().encode(
    x=alt.X("Year:T", title=""),
    y=alt.Y("Value:Q", title="", axis=alt.Axis(format="%", labelExpr="datum.label + (datum.value == 0.35 ? ' of GDP ' : '')", labelAlign=alt.expr("datum.value == 0 ? 'right' : 'right'"))),
    color=alt.Color("Country Name:N", legend=None),
    tooltip=["Country Name", "Year", "Value"]
).properties(
    width=500,
    height=350,
)

end_label = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=5,
    fontSize=12,
    color='#36B7B4',
    fontWeight='normal',
).encode(
    x=alt.X("Year:T"),
    y=alt.Y("Value:Q"),
    text=alt.Text("end_label:N")
).transform_filter(
    alt.datum['end_label'] != ''
)

chart = (chart + end_label).properties(
    width=500,
    height=350,
)

chart.save("charts/lebanon_remittances.png", scale_factor=3)
chart.save("charts/lebanon_remittances.json")
chart.display()



alt.LayerChart(...)

In [11]:
# Load dataset
df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")

# Focus on 2023 and your target countries
df = df.query("TIME_PERIOD == 2023")
with open("countries.txt", "w") as f:
    f.write("\n".join(df["COUNTRY"].unique()))



/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_95849/4222705904.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")


In [12]:
df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")

# Focus on 2023 and your target countries
df = df.query("TIME_PERIOD == 2023")
targets = ["Botswana", "Armenia, Republic of", "Lebanon"]

df.INDICATOR.unique()

/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_95849/3361941470.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")


array(["Reserves minus liabilities constituting foreign authorities' reserves (LCFAR), Adjusted internally by Fund staff",
       'Secondary income, Credit/Revenue',
       'Financial derivatives (other than reserves) and employee stock options, Net acquisition of financial assets, Assets',
       'Total capital account debit/expenditure, Debit/Expenditure',
       'Goods, Credit/Revenue',
       'Financial account balance (assets less liabilities), Net (net acquisition of financial assets less net incurrence of liabilities), Adjusted using IMF accounting records',
       'Services balance (credit less debit), Net (credits less debits)',
       'Financial derivatives (other than reserves) and employee stock options, Net incurrence of liabilities, Liabilities',
       'Services, Credit/Revenue',
       'International investment position, Assets, Adjusted using IMF accounting records',
       'Direct investment, Net incurrence of liabilities, Liabilities',
       'Total current account d

# UK and US primary income over time

In [13]:
import pandas as pd
import altair as alt

# Load IMF data
df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")

# Filter for primary and secondary income indicators
income_inds = {
    "Primary income": ["Primary income, Credit/Revenue", "Primary income, Debit/Expenditure"],
    "Secondary income": ["Secondary income, Credit/Revenue", "Secondary income, Debit/Expenditure"]
}
all_inds = [i for sublist in income_inds.values() for i in sublist]

df = df[df["INDICATOR"].isin(all_inds)]
df = df[df["COUNTRY"].isin(["United States", "United Kingdom"])]

# Pivot credit and debit
df_wide = df.pivot_table(index=["COUNTRY", "TIME_PERIOD", "INDICATOR"],
                         values="OBS_VALUE").reset_index()
df_wide["Account"] = df_wide["INDICATOR"].apply(lambda x: "Primary income" if "Primary" in x else "Secondary income")
df_wide["Flow"] = df_wide["INDICATOR"].apply(lambda x: "Credit" if "Credit" in x else "Debit")

df_pivot = df_wide.pivot(index=["COUNTRY", "TIME_PERIOD", "Account"], columns="Flow", values="OBS_VALUE").reset_index()
df_pivot["Net"] = df_pivot["Credit"] - df_pivot["Debit"]

# Load GDP data
gdp = pd.read_csv("API_NY/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_26433.csv", skiprows=4)
gdp = gdp[gdp["Country Name"].isin(["United States", "United Kingdom"])]

# Tidy GDP data
gdp_melted = gdp.melt(id_vars=["Country Name"], value_vars=[str(y) for y in range(2000, 2024)],
                      var_name="TIME_PERIOD", value_name="GDP")
gdp_melted.columns = ["COUNTRY", "TIME_PERIOD", "GDP"]
gdp_melted["TIME_PERIOD"] = gdp_melted["TIME_PERIOD"].astype(int)

# Merge with income data
df = df_pivot.merge(gdp_melted, on=["COUNTRY", "TIME_PERIOD"], how="left")
df["Pct of GDP"] = df["Net"] / df["GDP"]

df["TIME_PERIOD"] = pd.to_datetime(df["TIME_PERIOD"].astype(str), format="%Y")

df['end_label'] = np.where(df['TIME_PERIOD'] == df['TIME_PERIOD'].max(), df.COUNTRY, "")

# Filter and set up charts
base = alt.Chart(df).encode(
    x=alt.X("TIME_PERIOD:T", title=""),
    y=alt.Y("Pct of GDP:Q", title="", 
            scale=alt.Scale(domain=[-0.025, 0.015]),
            axis=alt.Axis(format=",.1%", labelExpr="datum.label + (datum.value == 0.015 ? ' of GDP ' : '')",)),
    color=alt.Color("COUNTRY:N", title="Country"),
    tooltip=["COUNTRY", "TIME_PERIOD", alt.Tooltip("Pct of GDP:Q", format=".2%")]
)

# Primary income chart
primary_chart = base.transform_filter(
    alt.datum.Account == "Primary income"
).mark_line(point=True).properties(
    title=alt.TitleParams(
        text="Primary Income",
        color="#676A86",
        anchor="start",
        fontWeight="normal",
        frame="group",
        fontSize=11,
        dx=0,
    ),
    width=300,
    height=300
)

# Secondary income chart
secondary_chart = base.transform_filter(
    alt.datum.Account == "Secondary income"
)

secondary_chart = secondary_chart.mark_line(point=True)

# end label
end_label = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=alt.expr("datum.COUNTRY == 'United States' ? -3 : 3"),
    fontSize=12,
    fontWeight='normal',
).encode(
    x=alt.X("TIME_PERIOD:T"),
    y=alt.Y("Pct of GDP:Q"),
    text=alt.Text("end_label:N"),
    color=alt.Color("COUNTRY:N", title="Country", legend=None),
).transform_filter(
    "datum.end_label != '' & datum.Account == 'Secondary income'"
)
secondary_chart = secondary_chart + end_label



secondary_chart = secondary_chart.properties(
    title=alt.TitleParams(
        text="Secondary Income",
        color="#676A86",
        fontWeight="normal",
        anchor="start",
        frame="group",
        fontSize=11,
        dx=0,
    ),
    width=300,
    height=300
)



# Combine side by side
combined_chart = alt.hconcat(primary_chart, secondary_chart).resolve_scale(
    y='independent'
)

combined_chart.save("charts/US_UK_primary_secondary_income.png", scale_factor=3)
combined_chart.save("charts/US_UK_primary_secondary_income.json")
combined_chart.display()


/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_95849/1987485130.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")


alt.HConcatChart(...)

In [14]:
secondary_chart.properties(
    title=alt.TitleParams(
        text="Secondary Income",
        color="#676A86",
        fontWeight="normal",
        anchor="start",
        frame="group",
        fontSize=11,
        dx=0,
    )
)

alt.LayerChart(...)

In [15]:
df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")


/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_95849/3923561128.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")


In [16]:
df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")

df.INDICATOR.unique()

/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_95849/2089014506.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset_2025-04-08T09_02_24.750240431Z_DEFAULT_INTEGRATION_IMF.STA_BOP_AGG_9.0.1.csv")


array(["Reserves minus liabilities constituting foreign authorities' reserves (LCFAR), Adjusted internally by Fund staff",
       'Secondary income, Credit/Revenue',
       'Financial derivatives (other than reserves) and employee stock options, Net acquisition of financial assets, Assets',
       'Total capital account debit/expenditure, Debit/Expenditure',
       'Goods, Credit/Revenue',
       'Financial account balance (assets less liabilities), Net (net acquisition of financial assets less net incurrence of liabilities), Adjusted using IMF accounting records',
       'Services balance (credit less debit), Net (credits less debits)',
       'Financial derivatives (other than reserves) and employee stock options, Net incurrence of liabilities, Liabilities',
       'Services, Credit/Revenue',
       'International investment position, Assets, Adjusted using IMF accounting records',
       'Direct investment, Net incurrence of liabilities, Liabilities',
       'Total current account d

# National Stats Agencies: UK and US

In [66]:
# ons_base = "https://api.beta.ons.gov.uk/v1/data?uri="

# # primary income
# req = requests.get(ons_base + "/economy/nationalaccounts/balanceofpayments/timeseries/d28m/ukea")
# data = json.loads(req.text)['years']
# primary_df = pd.DataFrame(data)
# primary_df['component'] = "Primary income"

# # secondary income
# req = requests.get(ons_base + "/economy/nationalaccounts/balanceofpayments/timeseries/d28n/pnbp")
# data = json.loads(req.text)['years']
# secondary_df = pd.DataFrame(data)
# secondary_df['component'] = "Secondary income"

# df = pd.concat([primary_df, secondary_df], ignore_index=True)
# df["country"] = "United Kingdom"
# df = df[["country", "component", "year", "value"]]
# df = df.rename(columns={"year": "date"})
# df.to_csv("uk_primary_secondary_income.csv", index=False)

uk_df = pd.read_csv("uk_primary_secondary_income.csv")
uk_df['date'] = pd.to_datetime(uk_df['date'], format="%Y")
uk_df['value'] = uk_df['value'].astype(float) / 100


# US

gdp_df = pd.read_csv('us_stats/US_nominal_gdp_billions_GDPA.csv')
gdp_df.columns = ['date', 'gdp']
gdp_df['gdp'] = gdp_df['gdp'].astype(float) * 1000

primary_df = pd.read_csv("us_stats/primary_income_balance_usd_millions_IEABCPIA.csv")
primary_df.columns = ['date', 'nominal']
primary_df['component'] = "Primary income"

secondary_df = pd.read_csv("us_stats/secondary_income_balance_usd_millions_IEABCPIA.csv_IEABCSIA.csv")
secondary_df.columns = ['date', 'nominal']
secondary_df['component'] = "Secondary income"

df = pd.concat([primary_df, secondary_df], ignore_index=True)
df['country'] = "United States"
df = df.merge(gdp_df, on="date", how="left")
df['value'] = df['nominal'] / df['gdp']
df = df[["country", "component", "date", "value"]]

df['date'] = pd.to_datetime(df['date'])

df = pd.concat([df, uk_df], ignore_index=True)
df['end_label'] = np.where(df['date'] == df['date'].max(), df.country, "")

# ons_base = "https://api.beta.ons.gov.uk/v1/data?uri="

df = df.query("date >= '2004-12-31'")

base = alt.Chart(df).encode(
    x=alt.X("date:T", title=""),
    y=alt.Y("value:Q", title="", 
            scale=alt.Scale(domain=[-0.025, 0.015]),
            axis=alt.Axis(format=",.1%", labelExpr="datum.label + (datum.value == 0.015 ? ' of GDP ' : '')",)),
    color=alt.Color("country:N", title="Country"),
    tooltip=["country", "date", alt.Tooltip("value:Q", format=".2%")]
)

zero_rule = alt.Chart(pd.DataFrame([{"y": 0}])).mark_rule(
    strokeDash=[5, 5],
    size=1,
    color="#676A86").encode(
    y=alt.Y("y:Q", title="")
)

# Primary income chart
primary_chart = base.transform_filter(
    alt.datum.component == "Primary income"
).mark_line(point=True)


primary_chart = (primary_chart+zero_rule).properties(
    title=alt.TitleParams(
        text="Primary Income",
        color="#676A86",
        anchor="start",
        fontWeight="normal",
        frame="group",
        fontSize=11,
        dx=0,
    ),
    width=300,
    height=300
)

# Secondary income chart
secondary_chart = base.transform_filter(
    alt.datum.component == "Secondary income"
)

secondary_chart = secondary_chart.mark_line(point=True)

# end label
end_label = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=alt.expr("datum.country == 'United States' ? 3 : -3"),
    fontSize=12,
    fontWeight='normal',
).encode(
    x=alt.X("date:T"),
    y=alt.Y("value:Q"),
    text=alt.Text("end_label:N"),
    color=alt.Color("country:N", title="Country", legend=None),
).transform_filter(
    "datum.end_label != '' & datum.component == 'Secondary income'"
)
secondary_chart = secondary_chart + end_label + zero_rule

secondary_chart = secondary_chart.properties(
    title=alt.TitleParams(
        text="Secondary Income",
        color="#676A86",
        fontWeight="normal",
        anchor="start",
        frame="group",
        fontSize=11,
        dx=0,
    ),
    width=300,
    height=300
)

# Combine side by side
combined_chart = alt.hconcat(primary_chart, secondary_chart).resolve_scale(
    y='independent'
)

# add the image
combined_chart = alt.vconcat(
    image.encode(x=alt.value(650), y=alt.value(0)).properties(
        height=1,
    ),
    combined_chart,
)

combined_chart.save("charts/US_UK_primary_secondary_income.png", scale_factor=3)
combined_chart.save("charts/US_UK_primary_secondary_income.json")
combined_chart.display()


alt.VConcatChart(...)

In [84]:
df['end_label'] = np.where(df['date'] == df['date'].max(), df.component, "")


base = alt.Chart(df).encode(
    x=alt.X("date:T", title=""),
    y=alt.Y(
        "value:Q", 
        title="", 
        scale=alt.Scale(domain=[-0.025, 0.015]),
        axis=alt.Axis(
            format=",.1%",
            labelExpr="datum.label + (datum.value == 0.015 ? ' of GDP ' : '')"
        )
    ),
    color=alt.Color("component:N", title="Income Component"),  # << CHANGED HERE
    tooltip=["country", "component", "date", alt.Tooltip("value:Q", format=".2%")]
).mark_line(point=True)

# -----------------------------
# US chart: Primary + Secondary
# -----------------------------
us_chart = base.transform_filter(
    alt.datum.country == "United States"
).mark_line(point=True)

us_chart = (us_chart + zero_rule).properties(
    title=alt.TitleParams(
        text="United States",
        color="#676A86",
        fontWeight="bold",
        frame="group",
        fontSize=14,
        dx=0,
    ),
    width=500,
    height=300
)

# -----------------------------
# UK chart: Primary + Secondary
# -----------------------------
uk_chart = base.transform_filter(
    alt.datum.country == "United Kingdom"
).mark_line(point=True)


# Optional: add end labels (only for US here, adjust if needed)
us_end_labels = alt.Chart(df.query("country == 'United States'")
                          
                          ).mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=alt.expr("datum.country == 'United States' ? 3 : -3"),
    fontSize=12,
    fontWeight='normal',
).encode(
    x=alt.X("date:T"),
    y=alt.Y("value:Q"),
    text=alt.Text("end_label:N"),
    color=alt.Color("component:N", title="Country", legend=None),
).transform_filter(
    "datum.end_label != ''"
)

uk_end_labels = alt.Chart(df.query("country == 'United Kingdom'")
                          ).mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=alt.expr("datum.country == 'United Kingdom' ? 3 : -3"),
    fontSize=12,
    fontWeight='normal',
).encode(
    x=alt.X("date:T"),
    y=alt.Y("value:Q"),
    text=alt.Text("end_label:N"),
    color=alt.Color("component:N", title="Country", legend=None),
).transform_filter(
    "datum.end_label != ''"
)

us_chart = us_chart + us_end_labels

uk_chart = (uk_chart + zero_rule).properties(
    title=alt.TitleParams(
        text="United Kingdom",
        color="#676A86",
        fontWeight="bold",
        frame="group",
        fontSize=14,
        dx=0,
    ),
    width=500,
    height=300
)
uk_chart = uk_chart + uk_end_labels

# -----------------------------
# Combine vertically: US on top, UK below
# -----------------------------
combined_chart = alt.vconcat(
    us_chart,
    uk_chart
).resolve_scale(
    y='independent'
)

# Save & display
combined_chart.save("charts/US_UK_primary_secondary_income_stacked.png", scale_factor=3)
combined_chart.save("charts/US_UK_primary_secondary_income_stacked.json")
combined_chart.display()

alt.VConcatChart(...)

# Recreating ONS Chart - UK Primary income

In [136]:
df = pd.read_csv("ons_Figure_4__The_primary_income_deficit_decreased_in_Quarter_4_2024_.csv", skiprows=6)
df.columns = ["date"] + list(df.columns[1:])
def parse_quarter_date(quarter_str):
    quarter = quarter_str.split(" ")[0]
    year = quarter_str.split(" ")[1]
    month = {
        "Q1": "01",
        "Q2": "04",
        "Q3": "07",
        "Q4": "10"
    }[quarter]
    return f"{year}-{month}-01"
# df['date'] = df['date'].apply(parse_quarter_date)
df = df.melt(id_vars=["date"], var_name="component", value_name="value")

df['component'] = df['component'].replace({
    "Other investment": "Other",
})

df['date'] = df['date'].str[-4:] + " " + df.date


bars = alt.Chart(
    df.query("component != 'Total'")).mark_bar().encode(
    x=alt.X("date:O", title="", 
            axis=alt.Axis(
                labelExpr="slice(datum.label, 4)",
            )
    ),
    y=alt.Y("value:Q", title="", axis=alt.Axis(
        labelExpr="datum.value < 0 ? '-£' + abs(datum.value) + ' bn' : '£' + datum.value + ' bn'",
        format=",.0f")),
    color=alt.Color("component:N", title="", 
                legend=alt.Legend(orient="top"),
            
        scale=alt.Scale(
        domain=["Direct investment", "Portfolio investment", "Other", "Total"],
        range=["#36B7B4","#E6224B","#F4C245", "black"]
    )),
    )

balance_line = alt.Chart(df.query("component == 'Total'")).mark_line(
    color="black",
    point=alt.OverlayMarkDef(
        filled=True,
        color="black",
        size=50,
    ),
    size=2,
).encode(
    x=alt.X("date:O", title=""),
    y=alt.Y("value:Q", title="", axis=alt.Axis(format=",.0f")),
)

chart = (bars + balance_line).properties(
    width=500,
    height=300,
    title=alt.TitleParams(
        text="United Kingdom",
        color="#676A86",
        fontWeight="bold",
        frame="group",
        fontSize=14,
        dx=0,
    )
)

chart.save("charts/primary_income_disagg_ons_quarterly_nominal.png", scale_factor=3)
chart.save("charts/primary_income_disagg_ons_quarterly_nominal.json")

chart




alt.LayerChart(...)

# US Version

In [138]:
df = pd.read_csv("Table 4.1. U.S. International Transactions in Primary Income _Table.csv", skiprows=3)
df.columns = ['i', 'component'] + ["2022-01-01", "2022-04-01", "2022-07-01", "2022-10-01", "2023-01-01", "2023-04-01", "2023-07-01", "2023-10-01", "2024-01-01", "2024-04-01", "2024-07-01", "2024-10-01"]
df.columns = ['i', 'component'] + ["Q1 2022", "Q2 2022", "Q3 2022", "Q4 2022", "Q1 2023", "Q2 2023", "Q3 2023", "Q4 2023", "Q1 2024", "Q2 2024", "Q3 2024", "Q4 2024"]
df = df.drop(columns=["i"])
df = df.iloc[2:, :]
df

receipts_df = df.iloc[:22, :].copy()
receipts_df = receipts_df.melt(id_vars=["component"], var_name="date", value_name="receipts")
receipts_df.component.replace({"Primary income receipts (table 1.1, line 5)": "Total"}, inplace=True)
payments_df = df.iloc[22:, :].copy()
payments_df = payments_df.melt(id_vars=["component"], var_name="date", value_name="payments")
payments_df.component.replace({"Primary income payments (table 1.1, line 13)": "Total"}, inplace=True)

df = pd.merge(receipts_df, payments_df, on=["component", "date"], how="outer")
df = df.dropna(subset=['component'])
df.component = df.component.str.strip()


keep = [
    "Direct investment income",
    "Portfolio investment income", 
    "Other investment income",
    "Reserve asset income",
    "Compensation of employees",
    "Total"
]


df = df.query("component in @keep")
df["payments"] = df["payments"].astype(float)
df["receipts"] = df["receipts"].astype(float)
df['value'] = df['receipts'] - df['payments']

# group reserve asset income into other investment income
df["component"] = df["component"].replace({
    "Reserve asset income": "Other",
    "Other investment income": "Other",
    "Compensation of employees": "Other",

})
df = df.groupby(["component", "date"]).agg({
    "receipts": "sum",
    "payments": "sum",
    "value": "sum"
}).reset_index()


df['component'] = df['component'].str.replace("income", "")

df['date'] = df['date'].str[-4:] + " " + df.date

df['value'] = df['value'].astype(float)/1_000



bars = alt.Chart(
    df.query("component != 'Total'")).mark_bar().encode(
    x=alt.X("date:N", title="",
                        axis=alt.Axis(
                labelExpr="slice(datum.label, 4)",
            )
    ),
    y=alt.Y("value:Q", title="", axis=alt.Axis(
        labelExpr="datum.value < 0 ? '-$' + abs(datum.value) + ' bn' : '$' + datum.value + ' bn'",
        format=",.0f")),
    color=alt.Color("component:N", title="", 
                # legend=alt.Legend(
                #     legendY=-20,
                #     orient="top"),
                legend=None,
            
        scale=alt.Scale(
        domain=['Direct investment ', 'Portfolio investment ', 'Other', 'Total'],
        range=["#36B7B4","#E6224B","#F4C245", "black"]
    )),
    )

balance_line = alt.Chart(df.query("component == 'Total'")).mark_line(
    color="black",
    point=alt.OverlayMarkDef(
        filled=True,
        color="black",
        size=50,
    ),
    size=2,
).encode(
    x=alt.X("date:O", title=""),
    y=alt.Y("value:Q", title="", axis=alt.Axis(format=",.0f")),
)

chart = (bars + balance_line).properties(
    width=500,
    height=300,
    title=alt.TitleParams(
        text="United States",
        color="#676A86",
        fontWeight="bold",
        frame="group",
        fontSize=14,
        dx=0,
    )
)

chart.save("charts/primary_income_disagg_us_quarterly_nominal.png", scale_factor=3)
chart.save("charts/primary_income_disagg_us_quarterly_nominal.json")
chart

/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_6313/2973550667.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  receipts_df.component.replace({"Primary income receipts (table 1.1, line 5)": "Total"}, inplace=True)
/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_6313/2973550667.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the in

alt.LayerChart(...)

In [102]:
df.component.unique()

array(['Direct investment ', 'Other', 'Portfolio investment ', 'Total'],
      dtype=object)

In [106]:
df.component.unique()

array(['Direct investment ', 'Other', 'Portfolio investment ', 'Total'],
      dtype=object)

# Recreating ONS Chart - UK seconary income

In [48]:
pb = pd.read_excel("pb.xlsx")

In [58]:
codes = pb.iloc[:1, :]
codes = codes.T.reset_index()
codes.columns = ["name", "code"]

target = "secondary income"
codes[codes["name"].str.contains(target, case=False)].to_csv("codes.csv", index=False)

# UK Chart

In [48]:
# Compensation of employees	"Investment income,
# Direct investment [note 1]"	"Investment income,
# Portfolio investment,
# Earnings on equity and investment funds shares"	"Investment income,
# Portfolio investment,
# Earnings on debt securities"	"Investment income,
# Portfolio investment,
# Total portfolio investment"	"Investment income,
# Other investment"	"Investment income,
# Reserve assets"	"Investment income,
# Total investment income"	Other primary income [note 2]	Total
# KTMP	MU7E	HLZO	HLZP	HLZX	CGNA	HHCB	HMBM	MT5W	HMBP

ids = {
    "Compensation of employees": "KTMP",
    "Investment income, Direct investment [note 1]": "MU7E",
    "Investment income, Portfolio investment, Earnings on equity and investment funds shares": "HLZO",
    "Investment income, Portfolio investment, Earnings on debt securities": "HLZP",
    "Investment income, Portfolio investment, Total portfolio investment": "HLZX",
    "Investment income, Other investment": "CGNA",
    "Investment income, Reserve assets": "HHCB",
    "Investment income, Total investment income": "HMBM",
    "Other primary income [note 2]": "MT5W",
}

ons_base = "https://api.beta.ons.gov.uk/v1/data?uri="

uk_dfs = []

for k, v in ids.items():
    url = ons_base + f"/economy/nationalaccounts/balanceofpayments/timeseries/{v.lower()}/pb"
    print(url)
    req = requests.get(url)
    data = json.loads(req.text)['years']
    df = pd.DataFrame(data)
    df['component'] = k
    uk_dfs.append(df)

uk_df = pd.concat(uk_dfs, ignore_index=True)
uk_df['country'] = "United Kingdom"
uk_df

https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/ktmp/pb
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/mu7e/pb
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/hlzo/pb
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/hlzp/pb
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/hlzx/pb
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/cgna/pb
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/hhcb/pb
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/hmbm/pb
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/mt5w/pb


,date,value,label,year,month,quarter,sourceDataset,updateDate,component,country
0,1946,-20,1946,1946,,,PB,2015-10-30T00:00:00.000Z,Compensation of employees,United Kingdom
1,1947,-19,1947,1947,,,PB,2015-10-30T00:00:00.000Z,Compensation of employees,United Kingdom
2,1948,-20,1948,1948,,,PB,2015-10-30T00:00:00.000Z,Compensation of employees,United Kingdom
3,1949,-20,1949,1949,,,PB,2015-10-30T00:00:00.000Z,Compensation of employees,United Kingdom
4,1950,-21,1950,1950,,,PB,2015-10-30T00:00:00.000Z,Compensation of employees,United Kingdom
...,...,...,...,...,...,...,...,...,...,...
477,2019,-1155,2019,2019,,,PB,2020-10-30T00:00:00.000Z,Other primary income [note 2],United Kingdom
478,2020,-2311,2020,2020,,,PB,2022-10-31T00:00:00.000Z,Other primary income [note 2],United Kingdom
479,2021,14,2021,2021,,,PB,2022-10-31T00:00:00.000Z,Other primary income [note 2],United Kingdom
480,2022,0,2022,2022,,,PB,2023-10-31T00:00:00.000Z,Other primary income [note 2],United Kingdom


In [60]:
df = pd.concat(uk_dfs, ignore_index=True)
df['country'] = "United Kingdom"
df = df[["country", "date", "component" , "value"]]
df = df.rename(columns={"value": "nominal"})

component_map = {
    "Investment income, Direct investment [note 1]": "Direct investment",
    "Investment income, Portfolio investment, Earnings on equity and investment funds shares": "Portfolio investment",
    "Investment income, Portfolio investment, Earnings on debt securities": "Portfolio investment",
    "Investment income, Portfolio investment, Total portfolio investment": "Portfolio investment",
    "Investment income, Other investment": "Other investment",
    "Compensation of employees": "Other",
    "Investment income, Reserve assets": "Other",
    "Other primary income [note 2]": "Other",
    "Investment income, Total investment income": "Total"
}

# Apply the mapping and drop rows that aren't in it
df["category"] = df["component"].map(component_map)
df = df.dropna(subset=["category"])

# Pivot to get totals per category per date (or per country-date if needed)
# You can include 'country' in the groupby if disaggregated by country
grouped_df = df.groupby(["date", "category"]).agg({"nominal": "sum"}).reset_index()

# Get GDP
uk_gdp_data = requests.get("https://api.beta.ons.gov.uk/v1/data?uri=/economy/grossdomesticproductgdp/timeseries/abmi/pn2")
uk_gdp_data = json.loads(uk_gdp_data.text)['years']
uk_gdp_data = pd.DataFrame(uk_gdp_data)
uk_gdp_data['country'] = "United Kingdom"
uk_gdp_data['gdp'] = uk_gdp_data['value'].astype(float) / 1000
uk_gdp_data = uk_gdp_data[["country", "date", "gdp"]]

df = df.merge(uk_gdp_data, on=["country", "date"], how="inner")
df['value'] = df['nominal'].astype(float) / df['gdp']

df['date'] = pd.to_numeric(df['date'], errors='coerce')

# Chart it

base = alt.Chart(df.query("date > 2000 and component != 'Total'")).encode(
    x=alt.X("date:O", title=""),
    y=alt.Y("value:Q", title="", 
            scale=alt.Scale(domain=[-0.025, 0.015]),
            axis=alt.Axis(format=",.1%", labelExpr="datum.label + (datum.value == 0.015 ? ' of GDP ' : '')",)),
    color=alt.Color("component:N", title=""),
).mark_bar()

base

alt.Chart(...)

In [ ]:
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/KTMP/bb
https://api.beta.ons.gov.uk/v1/data?uri=/economy/nationalaccounts/balanceofpayments/timeseries/ktmp/bb

,date,value,label,year,month,quarter,sourceDataset,updateDate,component
0,1955,0.6,1955,1955,,,UKEA,2022-09-29T23:00:00.000Z,Primary income
1,1956,0.8,1956,1956,,,UKEA,2022-09-29T23:00:00.000Z,Primary income
2,1957,0.9,1957,1957,,,UKEA,2022-09-29T23:00:00.000Z,Primary income
3,1958,1.0,1958,1958,,,UKEA,2022-09-29T23:00:00.000Z,Primary income
4,1959,0.8,1959,1959,,,UKEA,2022-09-29T23:00:00.000Z,Primary income
...,...,...,...,...,...,...,...,...,...
142,2020,-1.4,2020,2020,,,PNBP,2024-09-29T23:00:00.000Z,Secondary income
143,2021,-0.8,2021,2021,,,PNBP,2024-09-29T23:00:00.000Z,Secondary income
144,2022,-0.9,2022,2022,,,PNBP,2023-03-30T23:00:00.000Z,Secondary income
145,2023,-0.7,2023,2023,,,PNBP,2024-09-29T23:00:00.000Z,Secondary income


In [207]:
data

{'years': [{'date': '1955',
   'value': '0.6',
   'label': '1955',
   'year': '1955',
   'month': '',
   'quarter': '',
   'sourceDataset': 'UKEA',
   'updateDate': '2022-09-29T23:00:00.000Z'},
  {'date': '1956',
   'value': '0.8',
   'label': '1956',
   'year': '1956',
   'month': '',
   'quarter': '',
   'sourceDataset': 'UKEA',
   'updateDate': '2022-09-29T23:00:00.000Z'},
  {'date': '1957',
   'value': '0.9',
   'label': '1957',
   'year': '1957',
   'month': '',
   'quarter': '',
   'sourceDataset': 'UKEA',
   'updateDate': '2022-09-29T23:00:00.000Z'},
  {'date': '1958',
   'value': '1.0',
   'label': '1958',
   'year': '1958',
   'month': '',
   'quarter': '',
   'sourceDataset': 'UKEA',
   'updateDate': '2022-09-29T23:00:00.000Z'},
  {'date': '1959',
   'value': '0.8',
   'label': '1959',
   'year': '1959',
   'month': '',
   'quarter': '',
   'sourceDataset': 'UKEA',
   'updateDate': '2022-09-29T23:00:00.000Z'},
  {'date': '1960',
   'value': '0.6',
   'label': '1960',
   'year